In [3]:
import cv2 #for image processing
import easygui #to open the filebox
import numpy as np #to store image
import imageio #to read image stored at particular path
import sys
import matplotlib.pyplot as plt
import os
from PIL import ImageTk, Image

In [33]:
def upload():
    imgpath = easygui.fileopenbox()
    return imgpath

In [34]:
# size
size = (960, 540)
def cartonize(imgpath):
    # read the image
    origImage = cv2.imread(imgpath,0)# already convert image to gray
    # check if image chosen
    if origImage is None:
        print('Can not find any image. Please type correct name')
        sys.exit()
    #resize
    resizedOrigImage = cv2.resize(origImage, size)
    
    
    # converting image
    grayScaleImage = cv2.cvtColor(origImage, cv2.COLOR_BAYER_BG2GRAY)
    resizedGrayScale = cv2.resize(grayScaleImage, size)
    
    #smoothening image
    smoothGrayScale = cv2.medianBlur(grayScaleImage, 5)
    resizedSmoothImage = cv2.resize(smoothGrayScale, size)
    
    #Retrieving the edges of an image
    getEdge = cv2.adaptiveThreshold(
        smoothGrayScale, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,9,9
    )
    ResizedEdgeImage = cv2.resize(getEdge, size)
    
    
    #preparing a mask image
    colorImage = cv2.bilateralFilter(origImage, 9, 300, 300)
    ResizeColorImage = cv2.resize(colorImage, size)
    
    # giving Cartoon Effect
    cartonImage = cv2.bitwise_and(colorImage, colorImage, mask = getEdge)
    ResizeCartonImage = cv2.resize(cartonImage, size)
    
    #finish, plotting our image
    images = [resizedOrigImage,resizedGrayScale,resizedSmoothImage,ResizedEdgeImage,ResizeColorImage,ResizeCartonImage]
    fig, ax = plt.subplots(3,2, figsize=(8,8), subplot_kw={'xticks': [], 'yticks':[]}, gridspec_kw=dict(hspace=.1, wspace=.8))
    for i, ax in enumerate(ax.flat):
        ax.imshow(images[i], cmap='gray')
    
    
    return ResizeCartonImage

In [43]:
# let's save our photos
def save(ResizeCartonImage, imgpath):
    newName = 'cartoonified_image'
    path1 = os.path.dirname(imgpath)
    extension = os.path.splitext(imgpath)[1]
    path = os.path.join(path1, newName+extension)
    cv2.imwrite(path, ResizeCartonImage)

In [44]:
def main():
    imagepath = upload()
    img = cartonize(imagepath)
    save(img, imagepath)

In [ ]:
main()